# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key raw:", (private_key_raw_object.to_string().hex()) + '\x01')

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key compressed hex: {public_key.to_string("compressed").hex()}")
print(f"Public key uncompressed hex: {public_key.to_string("uncompressed").hex()}")


Private key raw: 6cd94e313bb7c97d669f5585afae25b288338475169ad767a290981176ea99c0
Private key: 6cd94e313bb7c97d669f5585afae25b288338475169ad767a290981176ea99c0
Public key compressed hex: 020b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c
Public key uncompressed hex: 040b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c40c8d4959e8fb6d8fc5b01fb879eda14e1c4e41888f74e2078e5d4d56113e77a


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  5878d45032cac912a956e557023807ea948b13d74b249b1279f4d48fa2c3d389
RIPEMD-160:  f05a703b1426e77f83e1fba340ac8fcd129f03eb
Address checksum:  3867db23
Binary Bitcoin Address:  00f05a703b1426e77f83e1fba340ac8fcd129f03eb3867db23
Bitcoin address:  1NusWoxV34BpR7NCisNytttdMdCLVnb962


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
def encode_varint(i: int) -> bytes:
    """
    Encode a potentially very large integer into varint bytes. The length should be
    specified in little-endian.

    https://bitcoin.org/en/developer-reference#compactsize-unsigned-integers
    """
    if i < 253:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError("Integer is too large: %d" % i)

In [6]:
# Path: concepts_bitcoin_transactions.ipynb
import base64

private_key_raw_1 = private_key_raw_object
msg = b"Hello world!"
msg_size = encode_varint(len(msg))
message_hash = hashlib.sha256(hashlib.sha256(b'\x18Bitcoin Signed Message:\n' + msg_size + msg).digest()).digest()
print("Message hash:", message_hash.hex())

signature_ = private_key_raw_1.sign_deterministic(message_hash)
# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
# message = b"\x18Bitcoin Signed Message:\n" + msg_size + msg
# message_size = encode_varint(len(message))
# signature_ = private_key.sign_deterministic(hashlib.sha256(hashlib.sha256(message).digest()).digest())
# print("Signature:", signature_.hex())
print(f'Signature in b64: {base64.b64encode(bytes([28]) + signature_).decode('ascii')}')
print("Signature hex:", binascii.hexlify(signature_).decode('ascii'))

# Verify the signature
print(public_key.verify(signature_, message_hash))


Message hash: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature in b64: HAscw9M3Uly7WLdKaGtiggGbVDx9+G4cwBZ5RaFDg8Tn9pZFC5p025JoPryxqP9WFuuGIwV3iEkssldESo+KW/Q=
Signature hex: 0b1cc3d337525cbb58b74a686b6282019b543c7df86e1cc0167945a14383c4e7f696450b9a74db92683ebcb1a8ff5616eb8623057788492cb257444a8f8a5bf4
True


In [7]:
# Get the private key
# private_key_hex = 'your_private_key_here'
# private_key_bytes = bytes.fromhex(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# The message to sign
# message = 'your_message_here'

# Sign the message
signature = private_key_raw_1.sign(message_hash)
print(f"Signature: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([28]) + signature).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Print the signature in base64 format
# signature_b64 = base64.b64encode(signature).decode('ascii')
# signature_b64 = base64.b64decode(signature)
# print(f"Signature b64: {signature_b64}")

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw_1.get_verifying_key()
print(f"Public key: {public_key.to_string('compressed').hex()}")


Signature: f37f64f951dd26a97b909f35089ab95b09fa7bdd8f325c2c71f28e10391ea68adb3ad4a65f0073f16dc28f889583a05b3409ae278af62b1be82fe4505dfa64b1
Signature b64: HPN/ZPlR3Sape5CfNQiauVsJ+nvdjzJcLHHyjhA5HqaK2zrUpl8Ac/Ftwo+IlYOgWzQJrieK9isb6C/kUF36ZLE=
Public key: 020b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c


### Verify a message

In [8]:
# Use the public key to verify a message
# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Signature valid


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [9]:
# pip install pybitcoin

In [10]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [11]:
# pip install bitcoin-utils

In [12]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [13]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [14]:
# pip install python-bitcoinlib

In [15]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash().hex()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")
print(f'signature in b64: {base64.b64decode(signature_).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: Kx5VwoQ8ju3iBN8XokyiqJ1hMzcRHK4yWALVTsqzvTQePrcebXz4
Public key: 03381fc46c767d800f511a6367d060590e5e7df9b1d1fe30778685288eaef4f1bb
Bitcoin address: 1C7KunanffXNXC4HNDeZGT2Ewoq27hS1DA
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 49452b4574593875645857704c6f4d78475158666f77536c3865767147423643574d39574835793955646f6246507a5948756b65353850506556616b594657694f4e445856347059696f7a596e2b366772764e423537383d
signature in b64: 204f84b58f2e7575a92e83311905dfa304a5f1ebea181e8258cf561f9cbd51da1b14fcd81ee91ee7c3cf7956a46055a238d0d7578a588a8cd89feea0aef341e7bf
Signature valid? True


In [16]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: 6cd94e313bb7c97d669f5585afae25b288338475169ad767a290981176ea99c001
Bitcoin private key: 6cd94e313bb7c97d669f5585afae25b288338475169ad767a290981176ea99c001
bitcoin public not encoded: 020b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c
Bitcoin public key: 020b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c
Bitcoin address from top: 1NusWoxV34BpR7NCisNytttdMdCLVnb962
Bitcoin address from pubkey: 1NusWoxV34BpR7NCisNytttdMdCLVnb962


In [17]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature_.decode()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(bytes([32]) + signature_).hex()
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes.decode()}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: IE+EtY8udXWpLoMxGQXfowSl8evqGB6CWM9WH5y9UdobFPzYHuke58PPeVakYFWiONDXV4pYiozYn+6grvNB578=
Signature from top b64: 49456c464b305630575468315a46685863457876545868485556686d623364546244686c646e4648516a5a44563030355630673165546c565a473969526c423657556831613255314f4642515a565a6861316c4756326c50546b5259566a527757576c76656c6c754b7a5a6e636e5a4f516a55334f44303d
Signature bytes: IAgz8heDOb02VHPDs58ddJoWSPRtNvqF4DfOJpixE2qxLlBlOUrAfBdz+FNUh8wbz1KdK0G7jR/yG8kNlpY9vaQ=
signature bytes decoded: 200833f2178339bd365473c3b39f1d749a1648f46d36fa85e037ce2698b1136ab12e5065394ac07c1773f8535487cc1bcf529d2b41bb8d1ff21bc90d96963dbda4
Signature valid? True


In [18]:
# Get the private key
private_key_hex = private_key_raw.hex()
private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
print(f"Private key: {private_key.to_string().hex()}")

# Get the public key
public_key_hex = private_key.get_verifying_key().to_string('compressed').hex()
public_key_bytes = binascii.unhexlify(public_key_hex)
public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
print(f"Public key: {public_key.to_string('compressed').hex()}")

# Get the bitcoin address
bitcoin_address = P2PKHBitcoinAddress.from_pubkey(public_key_bytes)
print(f"Bitcoin address: {bitcoin_address}")

# The message to sign
message = bitcoin_message_bytes.GetHash()
print(f"Message: {message.hex()}")

# Sign the message
signature = private_key.sign(message)

# Print the signature in hexadecimal format
print(f"Signature: {binascii.hexlify(signature).decode()}")
print(f"Signature ascii: {binascii.hexlify(signature).decode('ascii')}")

signature_hex = binascii.hexlify(signature).decode('ascii')

# The signature
signature_unhex = binascii.unhexlify(signature_hex.encode('ascii'))
print(f"Signature unhex: {signature_unhex}")

# signature from b64
# signature_from_b64 = base64.b64decode(signature_hex)
# print(f"Signature from b64: {signature_from_b64}")

# signature base64
signature_b64 = base64.b64encode(bytes([32]) + signature_unhex).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Verify the signature
try:
    public_key.verify(signature_unhex, message)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Private key: 6cd94e313bb7c97d669f5585afae25b288338475169ad767a290981176ea99c0
Public key: 020b45eebda7dc64dbd41381b49c5dd228ccd6e39d93ef4b0c222e714c6aaf921c
Bitcoin address: 1NusWoxV34BpR7NCisNytttdMdCLVnb962
Message: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: b1490c7eaa1a7bba98efa4e3df0650171cde873969062282e792b3c7eab3e029e12adf3af28e2c9292852d8f403cda27503f5d87860bea8e8a943e06f18e5997
Signature ascii: b1490c7eaa1a7bba98efa4e3df0650171cde873969062282e792b3c7eab3e029e12adf3af28e2c9292852d8f403cda27503f5d87860bea8e8a943e06f18e5997
Signature unhex: b'\xb1I\x0c~\xaa\x1a{\xba\x98\xef\xa4\xe3\xdf\x06P\x17\x1c\xde\x879i\x06"\x82\xe7\x92\xb3\xc7\xea\xb3\xe0)\xe1*\xdf:\xf2\x8e,\x92\x92\x85-\x8f@<\xda\'P?]\x87\x86\x0b\xea\x8e\x8a\x94>\x06\xf1\x8eY\x97'
Signature b64: ILFJDH6qGnu6mO+k498GUBcc3oc5aQYigueSs8fqs+Ap4SrfOvKOLJKShS2PQDzaJ1A/XYeGC+qOipQ+BvGOWZc=
Signature valid


In [19]:
# pip install pybitcoin

In [20]:
# !pip install bitcoin

# from pybitcoin import privtopub, ecdsa_raw_sign, encode_sig

# # Get the private key and message
# private_key = 'your_private_key_here'
# message = 'your_message_here'

# # Get the public key
# public_key = privtopub(private_key)

# # Sign the message
# r, s = ecdsa_raw_sign(message, private_key)

# # Calculate the recovery id
# recovery_id = 27 + (public_key[64] % 2)

# # Encode the signature in compact format
# compact_signature = encode_sig(recovery_id, r, s)

# print(f"Compact signature: {compact_signature}")

In [21]:
import hashlib
import ecdsa
import binascii
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage

def sign_message(private_key_hex, message):
    private_key_bytes = binascii.unhexlify(private_key_hex)
    private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    signature = private_key.sign(magic_message_hash)
    print(f"Signature: {signature}")
    # return binascii.hexlify(signature).decode()
    return signature

private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print(f"Private key: {private_key.to_string()}")

private_key_hex = private_key.to_string().hex()
print(f"Private key hex: {private_key_hex}")

message = 'Hello world!'
print(f"Message: {message}")

signature = sign_message(private_key_hex, message)
print(f"Signature hex: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([32]) + signature).decode()
print(f"Signature b64: {signature_b64}")

# Verify the signature
def verify_signature(public_key_hex, signature, message):
    public_key_bytes = binascii.unhexlify(public_key_hex)
    public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    return public_key.verify(signature, magic_message_hash)

public_key = private_key.get_verifying_key()
public_key_formatted = '04' + public_key.to_string().hex()
print(f"Public key formatted: {'04' + public_key.to_string().hex()}")

public_key_hex = public_key.to_string('compressed').hex()
print(f"Public key compressed hex: {public_key_hex}")

public_key_uncompressed = public_key.to_string('uncompressed').hex()
print(f"Public key uncompressed: {public_key_uncompressed}")

print(f"Signature valid (Check from pubkey)? {verify_signature(public_key_hex, signature, message)}")

# verify the signature using the python-bitcoinlib library
bitcoin_message = BitcoinMessage(message)
print(f"Bitcoin message: {bitcoin_message.magic} {bitcoin_message.message} {bitcoin_message.GetHash().hex()}")

bitcoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_hex), 0)
print(f"Bitcoin private key: {bitcoin_private_key}")
print(f"Bitcoin private key Hex: {bitcoin_private_key.hex()}")

# bitcoin_public_from_bitcoin_private = bitcoin_private_key.pub

bitcoin_public_key = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(public_key_hex))
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f'bitcoin public key hex: {bitcoin_public_key.to_bytes().hex()}')
# print(f"Bitcoin public key Hex: {bitcoin_public_key.to_scriptPubKey().hex()}")

bitcoin_public_key = bitcoin_private_key.pub
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f"Bitcoin public key hex: {bitcoin_public_key.hex()}")

bitcoin_address = P2PKHBitcoinAddress.from_pubkey(bitcoin_public_key)
print(f"Bitcoin address: {bitcoin_address}")

bitcoin_signed_message = SignMessage(bitcoin_private_key, bitcoin_message)
print(f"Bitcoin signed message: {bitcoin_signed_message}")
print(f"Bitcoin signed message base64decode: {base64.b64decode(bitcoin_signed_message).hex()}")

print(f"Signature valid? {VerifyMessage(bitcoin_address, bitcoin_message, bitcoin_signed_message)}")


Private key: b'\xfd\x84\xef\x93\xfc\x1dO\xff\xb1\xc5\xde\xc9\x05HMJWL\x8a\x89[\xec\x1cm\xea\x96p\x08\xfbz#Z'
Private key hex: fd84ef93fc1d4fffb1c5dec905484d4a574c8a895bec1c6dea967008fb7a235a
Message: Hello world!
Signature: b'\xd8@C\x8dR\xe7\xadS\x97u\xadT9\xe0\xb44\xc7\xf8G\xf6\xa0\xe7\xba\xa2E\xa7~hn\xc4\xca\x8bh\xe1\xbe\x9e\x9f\xc2nt\xf7\x0f\x99\xf5d\x88\xa1\x905\x94\xc6=\x834\xc16b\x94~\xfe\xaf\xdf\xc8\xe8'
Signature hex: d840438d52e7ad539775ad5439e0b434c7f847f6a0e7baa245a77e686ec4ca8b68e1be9e9fc26e74f70f99f56488a1903594c63d8334c13662947efeafdfc8e8
Signature b64: INhAQ41S561Tl3WtVDngtDTH+Ef2oOe6okWnfmhuxMqLaOG+np/CbnT3D5n1ZIihkDWUxj2DNME2YpR+/q/fyOg=
Public key formatted: 04c3ce6db729ad81bc98166d6e45aa1e2111a5d31aad00dd01d4cf6f4e0902542301246093fdc6c4e4300e7e96356df5facaa233c86e8198afabf1f829a61f47a8
Public key compressed hex: 02c3ce6db729ad81bc98166d6e45aa1e2111a5d31aad00dd01d4cf6f4e09025423
Public key uncompressed: 04c3ce6db729ad81bc98166d6e45aa1e2111a5d31aad00dd01d4cf6f4e0902542